In [39]:
import numpy as np
import pandas as pd
from pathlib import Path

example = """
.#..#
.....
#####
....#
...##
"""

input_text = Path("input.txt").read_text()

In [2]:
def text_to_array(text):
    return np.array([list(row.strip()) for row in text.strip().splitlines()])

text_to_array(example)

array([['.', '#', '.', '.', '#'],
       ['.', '.', '.', '.', '.'],
       ['#', '#', '#', '#', '#'],
       ['.', '.', '.', '.', '#'],
       ['.', '.', '.', '#', '#']], dtype='<U1')

In [3]:
def get_asteroids(arr):
    return np.flip(np.argwhere(arr == "#"), axis=1)

get_asteroids(text_to_array(example))

array([[1, 0],
       [4, 0],
       [0, 2],
       [1, 2],
       [2, 2],
       [3, 2],
       [4, 2],
       [4, 3],
       [3, 4],
       [4, 4]])

In [23]:
def get_angles(dxs, dys):
    degrees = np.rad2deg(np.arctan2(dys, dxs))
    degrees[degrees < 0] += 360.0
    return np.mod(degrees + 90.0, 360.0)

def get_dists(dxs, dys):
    return np.sqrt(dxs ** 2 + dys **2)

def filter_location(location, asteroids):
    return asteroids[np.abs(asteroids - location).sum(axis=1) > 0]
    
def line_of_sight(asteroids):
    result = np.zeros(len(asteroids), dtype=np.int)
    for i, candidate in enumerate(asteroids):
        others = filter_location(candidate, asteroids)
        dxs = others[:, 0] - candidate[0]
        dys = others[:, 1] - candidate[1]
        dists = get_dists(dxs, dys)
        angles = get_angles(dys, dxs)
        df = pd.DataFrame({"dist": dists, "angle": angles})
        result[i] = df.sort_values("dist").drop_duplicates("angle", keep="first").shape[0]
    return result

line_of_sight(get_asteroids(text_to_array(example)))

array([7, 7, 6, 7, 7, 7, 5, 7, 8, 7])

### Part 1

In [24]:
def get_best(text):
    arr = text_to_array(text)
    asteroids = get_asteroids(arr)
    n_in_sight = line_of_sight(asteroids)
    best_asteroid = asteroids[n_in_sight.argmax()]
    best_number = n_in_sight.max()
    return tuple(best_asteroid), best_number

In [25]:
(x, y), n = get_best("""
.#..#
.....
#####
....#
...##
""")
assert((x, y) == (3, 4))
assert(n == 8)

In [26]:
(x, y), n = get_best("""
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####
""")
assert((x, y) == (5, 8))
assert(n == 33)

In [27]:
(x, y), n = get_best("""
.#..#..###
####.###.#
....###.#.
..###.##.#
##.##.#.#.
....###..#
..#.#..#.#
#..#.#.###
.##...##.#
.....#.#..
""")
assert((x, y) == (6, 3))
assert(n == 41)

In [28]:
large_example = """
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
"""
(x, y), n = get_best(large_example)
assert((x, y) == (11, 13))
assert(n == 210)

In [38]:
(x, y), n = get_best(input_text)
print(f"best: {n} @ {x},{y}")

NameError: name 'INPUT' is not defined

### Part 2

In [61]:
def vaporize(text):
    arr = text_to_array(text)
    asteroids = get_asteroids(arr)
    (row, col), n = get_best(text)
    location = np.array([row, col])
    to_destroy = filter_location(location, asteroids)
    dys = to_destroy[:, 1] - location[1]
    dxs = to_destroy[:, 0] - location[0]
    dists = get_dists(dxs, dys)
    angles = get_angles(dxs, dys)
    df = pd.DataFrame({
        "dx": dxs,
        "dy": dys,
        "x": to_destroy[:, 0],
        "y": to_destroy[:, 1],
        "angle": get_angles(dxs, dys),
        "dist": get_dists(dxs, dys)
    }).sort_values(["angle", "dist"])
    i = 1
    n_killed = 1
    all_killed = df.loc[df.index < 0]
    while df.shape[0]:
        killed = df.drop_duplicates("angle", keep="first")
        for row in killed.itertuples():
            n_killed += 1
        print(f"--- round {i} finished; killed {len(killed)} ---")
        df = df.loc[~df.index.isin(killed.index)]
        all_killed = pd.concat([all_killed, killed])
        i += 1
    all_killed = all_killed.reset_index(drop=True)
    all_killed.index += 1
    return all_killed

def part2(text):
    result = vaporize(text)
    vaporized200 = result.loc[200]
    return vaporized200.x * 100 + vaporized200.y, result

#### Verify based on example

In the large example above (the one with the best monitoring station location at 11,13):

```
The 1st asteroid to be vaporized is at 11,12.
The 2nd asteroid to be vaporized is at 12,1.
The 3rd asteroid to be vaporized is at 12,2.
The 10th asteroid to be vaporized is at 12,8.
The 20th asteroid to be vaporized is at 16,0.
The 50th asteroid to be vaporized is at 16,9.
The 100th asteroid to be vaporized is at 10,16.
The 199th asteroid to be vaporized is at 9,6.
The 200th asteroid to be vaporized is at 8,2.
The 201st asteroid to be vaporized is at 10,9.
The 299th and final asteroid to be vaporized is at 11,1.
```

In [62]:
answer, result = part2(large_example)
print("answer:", answer)
result.loc[[1, 2, 3, 10, 20, 50, 100, 199, 200, 201, 299], :]

--- round 1 finished; killed 210 ---
--- round 2 finished; killed 43 ---
--- round 3 finished; killed 17 ---
--- round 4 finished; killed 8 ---
--- round 5 finished; killed 6 ---
--- round 6 finished; killed 5 ---
--- round 7 finished; killed 3 ---
--- round 8 finished; killed 2 ---
--- round 9 finished; killed 2 ---
--- round 10 finished; killed 1 ---
--- round 11 finished; killed 1 ---
--- round 12 finished; killed 1 ---
answer: 802.0


,dx,dy,x,y,angle,dist
1,0,-1,11,12,0.000000,1.000000
2,1,-12,12,1,4.763642,12.041595
3,1,-11,12,2,5.194429,11.045361
10,1,-5,12,8,11.309932,5.099020
20,5,-13,16,0,21.037511,13.928388
50,5,-4,16,9,51.340192,6.403124
100,-1,3,10,16,198.434949,3.162278
199,-2,-7,9,6,344.054604,7.280110
200,-3,-11,8,2,344.744881,11.401754
201,-1,-4,10,9,345.963757,4.123106


#### Part 2 answer

In [64]:
answer, _ = part2(input_text)
answer

--- round 1 finished; killed 303 ---
--- round 2 finished; killed 41 ---
--- round 3 finished; killed 10 ---
--- round 4 finished; killed 5 ---
--- round 5 finished; killed 2 ---
--- round 6 finished; killed 1 ---


408.0